In [8]:
import pyshark
import pandas as pd
import threading
import requests

def packet_to_dict(pkt):
    packet_dict = {
        'time': pkt.sniff_time.timestamp(),
        'frame_number': pkt.frame_info.number,
        'protocol': pkt.highest_layer,
        'source': pkt.ip.src if hasattr(pkt, 'ip') else None,
        'destination': pkt.ip.dst if hasattr(pkt, 'ip') else None,
        'length': int(pkt.length),
        'ttl': int(pkt.ip.ttl) if hasattr(pkt, 'ip') else None,
        'source_port': int(pkt[pkt.transport_layer].srcport) if pkt.transport_layer else None,
        'destination_port': int(pkt[pkt.transport_layer].dstport) if pkt.transport_layer else None,
        'tcp_flags': int(pkt.tcp.flags, 16) if hasattr(pkt, 'tcp') else None,
        'dns_query': pkt.dns.qry_name if hasattr(pkt, 'dns') and hasattr(pkt.dns, 'qry_name') else None,
        'dns_response': [resp.addr for resp in pkt.dns.resp_addr.all_fields] if hasattr(pkt, 'dns') and hasattr(pkt.dns, 'resp_addr') else None,
        'http_request_method': pkt.http.request_method if hasattr(pkt, 'http') and hasattr(pkt.http, 'request_method') else None,
        'http_host': pkt.http.host if hasattr(pkt, 'http') and hasattr(pkt.http, 'host') else None,
        'http_user_agent': pkt.http.user_agent if hasattr(pkt, 'http') and hasattr(pkt.http, 'user_agent') else None,
        'ssl_version': pkt.ssl.version if hasattr(pkt, 'ssl') and hasattr(pkt.ssl, 'version') else None,
        'tls_handshake_type': pkt.tls.handshake_type if hasattr(pkt, 'tls') and hasattr(pkt.tls, 'handshake_type') else None,
        # Add more fields as needed
    }
    return packet_dict

def packet_callback(pkt, packets_df):
    if isinstance(packets_df, pd.DataFrame):
        packet_dict = packet_to_dict(pkt)
        return pd.concat([packets_df, pd.DataFrame([packet_dict])], ignore_index=True)
    else:
        print("Error: packets_df is not a DataFrame.")
        return None

import requests

def capture_packets():
    packets_df = pd.DataFrame(columns=['time', 'frame_number', 'protocol', 'source', 'destination', 
                                        'length', 'ttl', 'source_port', 'destination_port', 'tcp_flags'])
    # Start capturing packets
    capture = pyshark.LiveCapture(interface='en0', display_filter='tcp or dns or http or ssl or tls')
    capture.sniff(timeout=10)

    # Start a new thread for making the request
    def make_request():
        response = requests.get("https://www.washingtonpost.com")
        print(response.status_code)

    request_thread = threading.Thread(target=make_request)
    request_thread.start()

    # Iterate over captured packets and store them in DataFrame
    for packet in capture.sniff_continuously(packet_count=10):
        packets_df = packet_callback(packet, packets_df)
        if packets_df is None:
            break  # Stop capturing if DataFrame becomes invalid

    print(packets_df)

if __name__ == "__main__":
    # Create a separate thread for packet capture
    capture_thread = threading.Thread(target=capture_packets)
    capture_thread.start()
    capture_thread.join()

if __name__ == "__main__":
    # Create a separate thread for packet capture
    capture_thread = threading.Thread(target=capture_packets)
    capture_thread.start()
    capture_thread.join()


/var/folders/_v/2_7ylzwd1r140fnllwbxf9hr0000gn/T/ipykernel_22738/3712537362.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([packets_df, pd.DataFrame([packet_dict])], ignore_index=True)


           time frame_number protocol        source   destination length  ttl  \
0  1.711087e+09            1      DNS   172.20.10.1   172.20.10.2    162   64   
1  1.711087e+09            2      TCP   172.20.10.2  23.50.153.34     78   64   
2  1.711087e+09            3      TCP  23.50.153.34   172.20.10.2     66  252   
3  1.711087e+09            4      TCP   172.20.10.2  23.50.153.34     54   64   
4  1.711087e+09            5      TLS   172.20.10.2  23.50.153.34    571   64   
5  1.711087e+09            6      TCP  23.50.153.34   172.20.10.2     54  252   
6  1.711087e+09            7      TLS  23.50.153.34   172.20.10.2   1414   49   
7  1.711087e+09            8      TCP   172.20.10.2  23.50.153.34     54   64   
8  1.711087e+09            9      TCP  23.50.153.34   172.20.10.2   1414   49   
9  1.711087e+09           10      TLS  23.50.153.34   172.20.10.2   1348   49   

  source_port destination_port tcp_flags               dns_query dns_response  \
0          53            64

/var/folders/_v/2_7ylzwd1r140fnllwbxf9hr0000gn/T/ipykernel_22738/3712537362.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([packets_df, pd.DataFrame([packet_dict])], ignore_index=True)


           time frame_number protocol        source   destination length  ttl  \
0  1.711087e+09            1      TCP  23.50.153.34   172.20.10.2     66  252   
1  1.711087e+09            2      TCP   172.20.10.2  23.50.153.34     54   64   
2  1.711087e+09            3      TLS   172.20.10.2  23.50.153.34    571   64   
3  1.711087e+09            4      TCP  23.50.153.34   172.20.10.2     54  252   
4  1.711087e+09            5      TLS  23.50.153.34   172.20.10.2   1414   49   
5  1.711087e+09            6      TCP   172.20.10.2  23.50.153.34     54   64   
6  1.711087e+09            7      TCP  23.50.153.34   172.20.10.2   1414   49   
7  1.711087e+09            8      TLS  23.50.153.34   172.20.10.2   1348   49   
8  1.711087e+09            9      TCP   172.20.10.2  23.50.153.34     54   64   
9  1.711087e+09           10      TLS   172.20.10.2  23.50.153.34    134   64   

  source_port destination_port tcp_flags dns_query dns_response  \
0         443            50835        18 

200
